In [272]:
# if tkinter is not installed use !pip install tk
# https://www.tutorialspoint.com/how-to-install-tkinter-in-python
import csv
import tkinter as tk
from tkinter import messagebox, filedialog

In [273]:

class Employee:
    def __init__(self,id, name, age, salary, job_title):
        self.id = id
        self.name = name
        self.age = age
        self.salary = salary
        self.job_title = job_title
        self.left = None
        self.right = None
        self.parent = None
        self.color = "RED"

class EmployeeDatabaseRBTree:
    def __init__(self):
        self.root = None
        self.full =' '
        
    def add_employee(self,id, name, age, salary, job_title):
        employee = Employee(id,name, age, salary, job_title)
        if self.root is None:
            self.root = employee
            employee.color = "BLACK"
        else:
            node = self.root
            while node is not None:
                if employee.id < node.id:
                    if node.left is None:
                        node.left = employee
                        employee.parent = node
                        break
                    else:
                        node = node.left
                else:
                    if node.right is None:
                        node.right = employee
                        employee.parent = node
                        break
                    else:
                        node = node.right
            self.__fix_insert(employee)

    def __fix_insert(self, node):
        while node != self.root and node.parent.color == "RED":
            if node.parent == node.parent.parent.left:
                uncle = node.parent.parent.right
                if uncle is not None and uncle.color == "RED":
                    node.parent.color = "BLACK"
                    uncle.color = "BLACK"
                    node.parent.parent.color = "RED"
                    node = node.parent.parent
                else:
                    if node == node.parent.right:
                        node = node.parent
                        self.__rotate_left(node)
                    node.parent.color = "BLACK"
                    node.parent.parent.color = "RED"
                    self.__rotate_right(node.parent.parent)
            else:
                uncle = node.parent.parent.left
                if uncle is not None and uncle.color == "RED":
                    node.parent.color = "BLACK"
                    uncle.color = "BLACK"
                    node.parent.parent.color = "RED"
                    node = node.parent.parent
                else:
                    if node == node.parent.left:
                        node = node.parent
                        self.__rotate_right(node)
                    node.parent.color = "BLACK"
                    node.parent.parent.color = "RED"
                    self.__rotate_left(node.parent.parent)
        self.root.color = "BLACK"

    def remove_employee(self, employee_id):
        node = self.__search(employee_id)
        if node is None:
            return None
        if node.left is None or node.right is None:
            child = node.left if node.left is not None else node.right
            if child is not None:
                child.parent = node.parent
            if node.parent is None:
                self.root = child
            else:
                if node == node.parent.left:
                    node.parent.left = child
                else:
                    node.parent.right = child
            if node.color == "BLACK":
                if child is None or child.color == "BLACK":
                    self.__fix_delete(child, node.parent)
                else:
                    child.color = "BLACK"
        else:
            successor = self.__successor(node)
            node.id, node.name, node.age, node.salary, node.job_title = successor.id, successor.name, successor.age, successor.salary, successor.job_title
            self.__fix_delete(successor, successor.parent)

    def __fix_delete(self, node, parent):
        while node != self.root and (node is None or node.color == "BLACK"):
            if node == parent.left:
                sibling = parent.right
                if sibling.color == "RED":
                    sibling.color = "BLACK"
                    parent.color = "RED"
                    self.__rotate_left(parent)
                    sibling = parent.right
                if (sibling.left is None or sibling.left.color == "BLACK") and (sibling.right is None or sibling.right.color == "BLACK"):
                    sibling.color = "RED"
                    node = parent
                    parent = node.parent
                else:
                    if sibling.right is None or sibling.right.color == "BLACK":
                        sibling.left.color = "BLACK"
                        sibling.color = "RED"
                        self.__rotate_right(sibling)
                        sibling = parent.right
                    sibling.color = parent.color
                    parent.color = "BLACK"
                    sibling.right.color = "BLACK"
                    self.__rotate_left(parent)
                    node = self.root
            else:
                sibling = parent.left
                if sibling.color == "RED":
                    sibling.color = "BLACK"
                    parent.color = "RED"
                    self.__rotate_right(parent)
                    sibling = parent.left
                if (sibling.right is None or sibling.right.color == "BLACK") and (sibling.left is None or sibling.left.color == "BLACK"):
                    sibling.color = "RED"
                    node = parent
                    parent = node.parent
                else:
                    if sibling.left is None or sibling.left.color == "BLACK":
                        sibling.right.color = "BLACK"
                        sibling.color = "RED"
                        self.__rotate_left(sibling)
                        sibling = parent.left
                    sibling.color = parent.color
                    parent.color = "BLACK"
                    sibling.left.color = "BLACK"
                    self.__rotate_right(parent)
                    node = self.root
        if node is not None:
            node.color = "BLACK"
    
    def __successor(self, node):
        if node.right is not None:
            node = node.right
            while node.left is not None:
                node = node.left
            return node
        parent = node.parent
        while parent is not None and node == parent.right:
            node = parent
            parent = parent.parent
        return parent
    
    def __rotate_left(self, node):
        right = node.right
        node.right = right.left
        if right.left is not None:
            right.left.parent = node
        right.parent = node.parent
        if node.parent is None:
            self.root = right
        else:
            if node == node.parent.left:
                node.parent.left = right
            else:
                node.parent.right = right
        right.left = node
        node.parent = right

    def __rotate_right(self, node):
        left = node.left
        node.left = left.right
        if left.right is not None:
            left.right.parent = node
        left.parent = node.parent
        if node.parent is None:
            self.root = left
        else:
            if node == node.parent.right:
                node.parent.right = left
            else:
                node.parent.left = left
        left.right = node
        node.parent = left

    def __search(self, employee_id):
        node = self.root
        while node is not None:
            if employee_id < node.id:
                node = node.left
            elif employee_id > node.id:
                node = node.right
            else:
                return node
        return None
    
    def __traverse_in_order(self):
        yield from self.__traverse_in_order_helper(self.root)

    def __traverse_in_order_helper(self, node):
        if node is not None:
            yield from self.__traverse_in_order_helper(node.left)
            yield node
            yield from self.__traverse_in_order_helper(node.right)


    def get_employee(self, employee_id):
        node = self.__search(employee_id)
        if node is None:
            return None
        return node
    
    def get_employees_by_name(self, name):
        matches = []
        for employee in self.__traverse_in_order():
            if employee.name == name:
                matches.append(employee)
        return matches

    def export_employees_csv(self, filename):
        with open(filename, mode='w', newline='') as csv_file:
            writer = csv.writer(csv_file)
            writer.writerow(["ID","Name","Age","Salary","Job Title"])
            for node in self.__traverse_in_order():
                writer.writerow([node.id, node.name, node.age, node.salary, node.job_title])

    def import_employees_csv(self, filename):
        with open(filename, mode='r') as csv_file:
            reader = csv.DictReader(csv_file)
            for row in reader:

                #reading id of the employee from given csv dataset
                emp_id = row['ID']
                #reading name for employee from given csv dataset
                name = row['Name']
                #reading the age 
                age = int(row['Age'])
                #reading the salary
                salary = int(row['Salary'])
                #reading the job position/title 
                job_title = row['Job Title']
                #adding it to the tree database
                self.add_employee(emp_id,name, age, salary, job_title)
                
    def __printCall(self, current_node, ind, is_last):
        stack = []
        while True:
            if current_node is not None:
                self.full += f'{ind}  '
                if is_last:
                    self.full += "R---- "
                    ind += "     "
                else:
                    self.full += "L---- "
                    ind += "|    "

                node_color = "RED" if current_node.color == "RED" else  "BLACK"
                self.full += str(current_node.name) + "(" + node_color + ") " + "(" + current_node.id + ") \n"
                stack.append((current_node.right, ind, True)) 
                current_node = current_node.left
                is_last = False
            elif stack:
                current_node, ind, is_last = stack.pop()
            else:
#                 print("Break hit")
                break
        treeString=str(self.full)
        self.full =' '
#         print(type(treeString))
        return treeString
        
    # Function to call print
    def print_tree(self):
        return self.__printCall(self.root, "", True)
        

In [274]:
#tkinter uasage refrence https://docs.python.org/3/library/tkinter.html 
class EmployeeDatabaseGUI(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.master = master
        self.master.title("Employee Database Using Red-black Tree")
        

        
        # create the employee database object
        self.db = EmployeeDatabaseRBTree()

        # create the widgets
        self.id_label = tk.Label(self, text="ID:")
        self.id_entry = tk.Entry(self)
        self.name_label = tk.Label(self, text="Name:")
        self.name_entry = tk.Entry(self)
        self.age_label = tk.Label(self, text="Age:")
        self.age_entry = tk.Entry(self)
        self.salary_label = tk.Label(self, text="Salary:")
        self.salary_entry = tk.Entry(self)
        self.job_title_label = tk.Label(self, text="Job Title:")
        self.job_title_entry = tk.Entry(self)
        
        # buttons 
        self.add_button = tk.Button(self, text="Add Employee", command=self.add_employee)
        self.remove_button = tk.Button(self, text="Remove Employee", command=self.remove_employee)
        self.export_button = tk.Button(self, text="Export to CSV", command=self.export_to_csv)
        self.import_button = tk.Button(self, text="Import from CSV", command=self.import_from_csv)
        self.search_button = tk.Button(self, text="Search Employee By ID", command=self.search_employee)
        self.visualize_button =tk.Button(self, text="Visualize Tree", command=self.Visualize_Tree)
        
        # search area
        self.search_label = tk.Label(self, text="Search Results:")
        self.search_text = tk.Text(self, height=35, state="disabled")

        # layout the widgets or the inputs to take
        self.id_label.grid(row=0, column=0, sticky="e")
        self.id_entry.grid(row=0, column=1)
        
        self.name_label.grid(row=1, column=0, sticky="e")
        self.name_entry.grid(row=1, column=1)
        
        self.age_label.grid(row=2, column=0, sticky="e")
        self.age_entry.grid(row=2, column=1)
        
        self.salary_label.grid(row=3, column=0, sticky="e")
        self.salary_entry.grid(row=3, column=1)
        
        self.job_title_label.grid(row=4, column=0, sticky="e")
        self.job_title_entry.grid(row=4, column=1)
        
        
        #1 row of buttons 
        self.add_button.grid(row=5, column=0)
        self.remove_button.grid(row=5, column=1)
        self.visualize_button.grid(row=5,column=2)
        
        #2 row of buttons
        self.export_button.grid(row=6, column=0)
        self.import_button.grid(row=6, column=1)
        self.search_button.grid(row=6, column=2)
        
        # 3rd row of buttons        
        self.search_label.grid(row=8, column=0, sticky="w")
        self.search_text.grid(row=9, column=0, columnspan=3)
        
        #
        footer_label = tk.Label(root, text="Made With Love by Vikas Vashisht", bg="lightgray", fg="black")
        footer_label.pack(side=tk.BOTTOM, fill=tk.X)

    def add_employee(self):
        empID = self.id_entry.get().strip()
        name = self.name_entry.get().strip()
        age = self.age_entry.get().strip()
        salary = self.salary_entry.get().strip()
        job_title = self.job_title_entry.get().strip()

        if not empID or not name or not age or not salary or not job_title:
            messagebox.showerror("Error", "Please fill in all fields")
            return

        try:
            age = int(age)
            salary = float(salary)
        except ValueError:
            messagebox.showerror("Error", "Invalid age or salary")
            return

        self.db.add_employee(empID,name, age, salary, job_title)
        messagebox.showinfo("Success", f"Employee added successfully")
        
        self.clear_entries()
        
    def remove_employee(self):
        empID = self.id_entry.get().strip()

        if not empID:
            messagebox.showerror("Error", "Please Enter Employee ID")
            return

        employee = self.db.remove_employee(empID)
       
        messagebox.showinfo("Success", "Employee removed")
        

    def export_to_csv(self):
        file_path = filedialog.asksaveasfilename(defaultextension=".csv")

        if file_path:
            self.db.export_employees_csv(file_path)
            messagebox.showinfo("Success", "Data exported to CSV")

    def import_from_csv(self):
        file_path = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])

        if file_path:
            self.db.import_employees_csv(file_path)
            messagebox.showinfo("Success", "Data imported from CSV")
            
    def Visualize_Tree(self):
        tree_view=self.db.print_tree()
        self.search_text.configure(state="normal")
        self.search_text.delete("1.0", tk.END)
        
        self.search_text.insert(tk.END, f"Visualization of Tree :\n")
        self.search_text.insert(tk.END, "-"*20 + "\n\n")
        self.search_text.insert(tk.END, tree_view)
        
        self.search_text.insert(tk.END, "\n\n")
        self.search_text.insert(tk.END, "-"*20 + "\n")
        self.search_text.configure(state="disabled")
        
    def search_employee(self):
        emp_id=self.id_entry.get().strip()
        
        if not name:
            messagebox.showerror("Error", "Please enter a name to search")
            return

        employee = self.db.get_employee(employee_id=emp_id)
        if employee:
            self.search_text.configure(state="normal")
            self.search_text.delete("1.0", tk.END)
            
            self.search_text.insert(tk.END, f"ID: {employee.id}\n")
            self.search_text.insert(tk.END, f"Name: {employee.name}\n")
            self.search_text.insert(tk.END, f"Age: {employee.age}\n")
            self.search_text.insert(tk.END, f"Salary: {employee.salary}\n")
            self.search_text.insert(tk.END, f"Job Title: {employee.job_title}\n")
            self.search_text.insert(tk.END, "-"*20 + "\n")
            
            self.search_text.configure(state="disabled")
        else:
            messagebox.showinfo("Search Results", "No employees found")
            
    def clear_entries(self):
        self.id_entry.delete(0, tk.END)
        self.name_entry.delete(0, tk.END)
        self.age_entry.delete(0, tk.END)
        self.salary_entry.delete(0, tk.END)
        self.job_title_entry.delete(0, tk.END)
        


In [275]:
root = tk.Tk()
app = EmployeeDatabaseGUI(root)
app.pack()
root.mainloop()